<a href="https://colab.research.google.com/github/UNT-CSCE-Projects/US_stock_market_prediction/blob/dev_backend/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

def train_and_predict_models(df):
    # Filter rows where 'stock_id' is 22
    train_dataset_row_22 = df[df['stock_id'] == 22]

    # Drop unnecessary columns
    train_22 = train_dataset_row_22.drop(axis='columns', columns=['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'row_id'])

    # Set 'time_id' as the index
    train_22 = train_22.set_index('time_id')

    # SARIMAX model
    p, d, q, P, D, Q, s = 1, 1, 1, 1, 1, 1, 7 # Example values, please replace with appropriate orders and seasonal periodicity
    column_name = 'target' # Corrected column name
    model_sarimax = SARIMAX(train_22[column_name], order=(p, d, q), seasonal_order=(P, D, Q, s), max_iter=1000)
    result_sarimax = model_sarimax.fit()

    # ARIMA model
    day = 1
    start = (day - 1) * 55
    end = day * 55

    arma_diff1 = ARIMA(train_22[start:end-4].diff(), order=(1, 0, 3)).fit()
    forecast_arma = arma_diff1.predict(end-4, end-1, dynamic=True)
    test = train_22[end-4:end]
    mse_arma = ((forecast_arma - test[column_name]) ** 2).mean()
    baseline_mae_arma = (forecast_arma - test[column_name]).abs().mean()

    return result_sarimax, forecast_arma, test

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/train.csv')

# Call the function
result_sarimax, forecast_arma, test = train_and_predict_models(df)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['max_iter']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
